In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
# import numpy as np

In [ ]:
# rng = np.random.RandomState(1)
# x = 10 * rng.rand(50)
# y = 2 * x - 5 + rng.randn(50)
# plt.scatter(x, y);

Podemos usar o estimador ``LinearRegression`` do Scikit-Learn para ajustar esses dados e construir a linha de melhor ajuste, mas quem seria a variável alvo e quem seria o 'rotulo'?

In [ ]:
# from sklearn.linear_model import LinearRegression

# model = LinearRegression(fit_intercept=True)

# model.fit(... , ...) # x[:, np.newaxis], y

Agora para fazer um teste como podemos consultar esta função?

In [ ]:
# xfit = np.linspace(0, 10, 1000)
# yfit = model.predict(xfit[:, np.newaxis])

# plt.scatter(x, y)
# plt.plot(xfit, yfit,'--r');

A inclinação e o termo livre (a e b) estão contidas nos parâmetros de ajuste do modelo, que no Scikit-Learn são ``coef_`` e ``intercept_``:

In [ ]:
# print("Model slope:    ", model.coef_[0])
# print("Model intercept:", model.intercept_)

Regressão linear multipla

In [ ]:
# rng = np.random.RandomState(1)
# X = 10 * rng.rand(100, 3)
# y = 0.5 + np.dot(X, [1.5, -2., 1.])

# model.fit(X, y)
# print(model.intercept_)
# print(model.coef_)

Aqui, os dados $y$ são construídos a partir de três valores $x$ aleatórios, e a regressão linear recupera os coeficientes usados ​​para construir os dados.

Dessa forma, podemos usar o estimador único ``LinearRegression`` para ajustar retas, planos ou hiperplanos aos nossos dados.

Ainda parece que essa abordagem se limitaria a relações estritamente lineares entre variáveis, mas também podemos flexibilizar isso.

## Regressão com transformação de features

O truque que vamos usar para adaptar a regressão linear a relações não lineares entre variáveis ​​é transformar os dados de acordo com *funções base*.

A ideia é pegar nosso modelo linear multidimensional:

$$
y = a_0 + a_1 x_1 + a_2 x_2 + a_3 x_3 + \cdots
$$

e construir $x_1, x_2, x_3,$ e assim por diante, a partir de nossa entrada unidimensional $x$.

Ou seja, consideramos $x_n = f_n(x)$, onde $f_n()$ é uma função que transforma nossos dados.

Por exemplo, se $f_n(x) = x^n$, nosso modelo se torna uma regressão polinomial:

$$
y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots
$$

Observe que este *ainda é um modelo linear* — a linearidade se refere ao fato de que os coeficientes $a_n$ nunca se multiplicam ou se dividem.

O que efetivamente fizemos foi pegar nossos valores unidimensionais de $x$ e projetá-los em uma dimensão superior, de modo que um ajuste linear possa ajustar relações mais complexas entre $x$ e $y$.

### Polynomial basis functions

This polynomial projection is useful enough that it is built into Scikit-Learn, using the ``PolynomialFeatures`` transformer:

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures

# x = np.array([2, 3, 4])
# poly = PolynomialFeatures(3, include_bias=False)
# poly.fit_transform(x[:, None])

We see here that the transformer has converted our one-dimensional array into a three-dimensional array by taking the exponent of each value.
This new, higher-dimensional data representation can then be plugged into a linear regression.

As we saw in [Feature Engineering](05.04-Feature-Engineering.ipynb), the cleanest way to accomplish this is to use a pipeline.
Let's make a 7th-degree polynomial model in this way:

In [ ]:
# from sklearn.pipeline import make_pipeline

# poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())

With this transform in place, we can use the linear model to fit much more complicated relationships between $x$ and $y$.
For example, here is a sine wave with noise:

In [ ]:
# rng = np.random.RandomState(1)
# x = 10 * rng.rand(50)
# y = np.sin(x) + 0.1 * rng.randn(50)

# poly_model.fit(x[:, np.newaxis], y)
# yfit = poly_model.predict(xfit[:, np.newaxis])

# plt.scatter(x, y)
# plt.plot(...,...); # xfit, yfit

## Example: Predicting Bicycle Traffic

Como exemplo, vamos verificar se podemos prever o número de viagens de bicicleta pela Ponte Fremont, em Seattle, com base no clima, na estação do ano e em outros fatores.

Vamos juntar os dados das bicicletas a outro conjunto de dados e tentaremos determinar em que medida os fatores climáticos e sazonais — temperatura, precipitação e horas de luz do dia — afetam o volume de tráfego de bicicletas por este corredor.

Felizmente, a NOAA disponibiliza seus [dados diários das estações meteorológicas](http://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND) e podemos facilmente usar o Pandas para unir as duas fontes de dados.

Realizaremos uma regressão linear simples para relacionar informações meteorológicas e outras informações à contagem de bicicletas, a fim de estimar como uma mudança em qualquer um desses parâmetros afeta o número de ciclistas em um determinado dia.

In [ ]:
# !wget https://github.com/juancolonna/UEA-2025-ML-course/raw/main/datasets/BicycleWeather.csv
# !wget https://github.com/juancolonna/UEA-2025-ML-course/raw/main/datasets/FremontBridge.csv

In [ ]:
# import pandas as pd

# counts = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True, date_format='%m/%d/%Y %I:%M:%S %p')
# counts.sort_index(inplace=True)
# display(counts)

# weather = pd.read_csv('BicycleWeather.csv', index_col='DATE', parse_dates=True)
# display(weather)

Calcularemos o tráfego diário total de bicicletas e o colocaremos em seu próprio quadro de dados:

In [ ]:
# daily = counts.resample('d').sum()
# daily['Total'] = daily['Fremont Bridge Total']
# daily = daily[['Total']] # remove other columns
# daily

Vimos anteriormente que os padrões de uso geralmente variam de um dia para o outro; vamos levar isso em conta em nossos dados adicionando colunas binárias que indicam o dia da semana:

In [ ]:
# days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
# for i in range(7):
#     daily[days[i]] = (daily.index.dayofweek == i).astype(float)

# daily

Da mesma forma, podemos esperar que os passageiros se comportem de maneira diferente nos feriados; vamos adicionar um indicador disso também:

In [ ]:
# from pandas.tseries.holiday import USFederalHolidayCalendar

# cal = USFederalHolidayCalendar()
# holidays = cal.holidays('2012', '2021')
# daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
# daily['holiday'] = daily['holiday'].fillna(0)
# daily

Também podemos suspeitar que as horas de luz do dia afetariam o número de pessoas que andam de bicicleta; vamos usar o cálculo astronômico padrão para adicionar essas informações:

In [ ]:
# def hours_of_daylight(date, axis=23.44, latitude=47.61):
#     """Compute the hours of daylight for the given date"""
#     days = (date - pd.Timestamp(2021, 12, 21)).days # winter solstice
#     m = (1. - np.tan(np.radians(latitude)) * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
#     return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

# daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))
# daily[['daylight_hrs']].plot()
# plt.ylim(8, 17)

# daily

Também podemos adicionar a temperatura média e a precipitação total aos dados. Além dos centímetros de precipitação, vamos adicionar um sinalizador que indica se um dia está seco (com precipitação zero):

In [ ]:
# # temperatures are in 1/10 deg C; convert to C
# weather['TMIN'] /= 10
# weather['TMAX'] /= 10
# weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])

# # precip is in 1/10 mm; convert to inches
# weather['PRCP'] /= 254
# weather['dry day'] = (weather['PRCP'] == 0).astype(int)

# daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])
# daily['PRCP'] = daily['PRCP'].fillna(0)
# daily

In [ ]:
# daily['PRCP'] # tomar cuidado na interpretação, esta variável está em milĩmetros cúbicos

In [ ]:
# # verificar a existencia de algum valor faltante
# nan_check = daily.isnull()

# print("\nNumber of NaN values per column:")
# display(nan_check.sum())

In [ ]:
# Drop any rows with null values
# daily.dropna(axis=0, how='any', inplace=True) # não teoria não é necessário

Por fim, vamos adicionar um contador que aumenta a partir do primeiro dia e mede quantos anos se passaram.
Isso nos permitirá medir qualquer aumento ou diminuição anual observada nas travessias diárias:

In [ ]:
# daily['annual'] = (daily.index - daily.index[0]).days / 365.
# display(daily)

Com isso em mãos, podemos escolher as colunas a serem usadas e ajustar um modelo de regressão linear aos nossos dados.
Definiremos ``fit_intercept = False``, porque os sinalizadores diários operam essencialmente como suas próprias interceptações específicas do dia:

In [ ]:
# column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'holiday',
#                 'daylight_hrs', 'PRCP', 'dry day', 'Temp (C)', 'annual']

# X = daily[column_names]
# y = daily['Total']

# model = LinearRegression(fit_intercept=False)
# model.fit(.., ...)

# daily['predicted'] = model.predict(X)

In [ ]:
# daily[['Total', 'predicted']].plot(alpha=0.5, figsize=(25,5));

Duas coisas são evidentes:

* deixamos de lado algumas características importantes, especialmente durante o verão. Ou nossas características não estão completas (ou seja, as pessoas decidem se vão de bicicleta para o trabalho com base em mais do que apenas essas características) ou existem algumas relações não lineares que deixamos de levar em consideração (por exemplo, talvez as pessoas andem menos de bicicleta em temperaturas altas e baixas).

* lockdown pandemia a partir do ano 2020

No entanto, nossa aproximação aproximada é suficiente para nos dar alguns insights, e podemos analisar os coeficientes do modelo linear para estimar o quanto cada característica contribui para a contagem diária de bicicletas:

In [ ]:
# params = pd.Series(model.coef_, index=X.columns)
# params

Esses números são difíceis de interpretar sem alguma medida de sua incerteza.
Podemos calcular essas incertezas rapidamente usando reamostragens bootstrap dos dados:

In [ ]:
# from sklearn.utils import resample

# np.random.seed(1)

# err = np.std([model.fit(*resample(X, y)).coef_ for i in range(1000)], 0) # * unpack operator

Com esses erros estimados, vamos analisar novamente os resultados:

In [ ]:
# print(pd.DataFrame({'effect': params.round(0), 'error': err.round(0)}))

Primeiro, vemos que há uma tendência relativamente estável na linha de base semanal: há muito mais ciclistas durante a semana do que nos fins de semana e feriados.

Observamos que, para cada hora adicional de luz do dia, 150 ± 19 pessoas a mais optam por pedalar; um aumento de temperatura de um grau Celsius incentiva 732 ± 46 pessoas a pegarem suas bicicletas;

Um dia seco significa, em média, 1.023 ± 71 ciclistas a mais, e cada polegada de precipitação significa 340 ± 270 pessoas a mais que deixam suas bicicletas em casa.

Considerando todos esses efeitos, observamos uma redução modesta de 273 ± 13 ciclistas diários a cada ano (devido ao efeito da pandemia).

É quase certo que nosso modelo não contém alguma informação relevante. Por exemplo, efeitos não lineares (como os efeitos da precipitação *e* da temperatura fria) e tendências não lineares dentro de cada variável (como a relutância em pedalar em temperaturas muito frias e muito quentes) não podem ser considerados neste modelo.

Além disso, descartamos algumas das informações mais refinadas (como a diferença entre uma manhã chuvosa e uma tarde chuvosa) e ignoramos as correlações entre os dias (como o possível efeito de uma terça-feira chuvosa nos números de quarta-feira, ou o efeito de um dia ensolarado inesperado após uma sequência de dias chuvosos).

Todos esses são efeitos potencialmente interessantes, e agora você tem as ferramentas para começar a explorá-los, se desejar!

Aqui está uma breve explicação das métricas de avaliação de regressão que calculamos:

*   **Erro Médio Absoluto (MAE - Mean Absolute Error)**: Representa a média das diferenças absolutas entre os valores reais e as previsões do modelo. Ele nos dá uma ideia do erro típico que podemos esperar nas previsões, sem considerar a direção do erro (superestimação ou subestimação).

$$
MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

*   **Erro Quadrático Médio (MSE - Mean Squared Error)**: É a média dos erros quadrados. Ele penaliza erros maiores de forma mais significativa do que o MAE, pois os erros são elevados ao quadrado.
    
$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

*   **Raiz do Erro Quadrático Médio (RMSE - Root Mean Squared Error)**: É a raiz quadrada do MSE. Está na mesma unidade da variável que estamos tentando prever, o que o torna mais fácil de interpretar do que o MSE. Ele representa o desvio padrão dos resíduos (os erros de previsão).
$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

*   **R² (Coeficiente de Determinação)**: Indica a proporção da variância na variável de destino (tráfego total de bicicletas) que é explicada pelo modelo. Um valor de R² próximo de 1 sugere que o modelo explica uma grande parte da variância nos dados, enquanto um valor próximo de 0 indica que o modelo explica muito pouco.
$$
R^2 = 1 - \frac{MSE}{Var(y)} = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$
Onde $y_i$ são os valores reais, $\hat{y}_i$ são as previsões do modelo e $\bar{y}$ é a média dos valores reais.

In [ ]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# y_true = daily['Total']
# y_pred = daily['predicted']

# mae = mean_absolute_error(..., ...)
# mse = mean_squared_error(y_true, y_pred)
# rmse = np.sqrt(mse) # Calculate RMSE from MSE
# r2 = r2_score(y_true, y_pred)

# print(f"Mean Absolute Error (MAE): {mae:.2f}")
# print(f"Mean Squared Error (MSE): {mse:.2f}")
# print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
# print(f"R-squared (R²): {r2:.2f}")

Inclua sua tarefa a partir daqui